In [1]:
# Required packages, install if not installed (assume PyTorch* and Intel® Extension for PyTorch* is already present)
!echo "Installation in progress..."
# import sys
# !{sys.executable} -m pip install  invisible-watermark > /dev/null
# !conda install -y --quiet --prefix {sys.prefix}  -c conda-forge \
#     accelerate==0.23.0 \
#     validators==0.22.0 \
#     diffusers==0.18.2 \
#     transformers==4.32.1 \
#     tensorboardX \
#     pillow \
#     ipywidgets \
#     ipython > /dev/null && echo "Installation successful" || echo "Installation failed"
import sys
!{sys.executable} -m pip install invisible-watermark --user > /dev/null 2>&1
!{sys.executable} -m pip install openai --user > /dev/null 2>&1
#!{sys.executable} -m pip install transformers huggingface-hub --user > /dev/null 2>&1
!echo "Installtion complete..."

Installation in progress...
Installtion complete...


In [2]:
from io import BytesIO
import os
import random
import time
import warnings
from pathlib import Path
from typing import List, Dict, Tuple
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

import requests
import torch
import torch.nn as nn
import intel_extension_for_pytorch as ipex  # Used for optimizing PyTorch models
from PIL import Image

import ipywidgets as widgets
from IPython.display import clear_output
from IPython.display import Image as IPImage

from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
import openai as OpenAI
from PIL import Image
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler, StableDiffusionImg2ImgPipeline
import logging
logging.getLogger().setLevel(logging.CRITICAL)

OpenAI.api_key = ""
model_dir = "/home/common/data/Big_Data/GenAI"

In [3]:
def get_completion(prompt, text, model = "gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = ""
    try:
        response = OpenAI.chat.completions.create(model = model, messages = messages, temperature = 0,)
    except:
        print("\n\nOpenAI LLM is not responding. Proper story can't be generated...")
        return text
    return response.choices[0].message.content


def get_prompt(input):
    message = f"""
    A part of a story will be provided to you and you have to generate a simple prompt that describes the \
    scenario in that part of the story such that the part of the story can be explained in an image generated by the prompt generated by you.
    
    Here are some rules u have to follow while generating the prompts: 
    1. The prompt be strictly less than 70 words.
    2. Don't include special characters other than comma and hyphen and dot.
    3. You just have to describe the scenario not write the whole story.
    4. Always include "A colored cartoon type sketch of," at the start of every prompt.
    5. The very important one, write in crisp and very simple english, don't use complicated words.
    6. Separate the different traits of the scenario with commas.
    7. If you can't understand the story or text, just write whatever you think the situation could be in the text.
    
    Here are some examples on how to generate the prompt:
    
    Example story paragraph:
    Once upon a time, in a not-so-distant future, there lived a man named Alex. Alex was an adventurous soul who dreamed of exploring the great \
    unknown: outer space. From a young age, he would gaze up at the stars with wonder, imagining what it would be like to journey among them.
    Expected text from you is:
    A colored cartoon type sketch of, a man looking up in the sky at night, sky has stars & moon.
    
    Example story paragraph:
    As the days turned into weeks, Maya forged friendships with the creatures of the jungle. She shared moments of laughter with mischievous monkeys, \
    and learned the ancient wisdom of wise old elephants. Together, they explored hidden caves and winding rivers, each new discovery fueling Maya's sense of wonder.
    Expected text from you is:
    A colored cartoon type sketch of, A girl laughing with monkeys, old elephants, hidden caves, winding rivers.
    
    Example story paragraph:
    In the heart of a bustling metropolis, where skyscrapers kissed the sky and streets hummed with the \
    rhythm of life, there existed a city like no other. Its streets were a labyrinth of winding alleys and bustling boulevards, \
    lined with towering buildings that reached for the clouds.
    Expected prompt generated from you is:
    A colored cartoon type sketch of, a metropolitan city, high skycrapers, streets, sky with clouds.
    
    Example story paragraph:
    what's up
    Since the paragraph is vague to understand, you can assume that a person is saying what's up to another person, for this the expected \
    text generated by you is:
    A colored cartoon type sketch of, two person speaking.
    
    Further rules:
    Please don't generate more than 70 words, this is a must.
    Please note that all the above examples the generated prompts were less than 20 words, you must also generate the prompts strictly less than 70 words.
    
    I just want the prompt from you not the explanation of why you generated that prompt.
    Now the actual story paragraph for which the prompt is to be generated is the text delimited by triple backticks
    Text:
    ```{input}```
    """
    return get_completion(message, input)

def get_story_from_plot(plot):
    message = f"""
    Write a creative story based on the plot provided in text delimited by triple backticks in \
    3000 words.
    Text:
    ```{plot}```
    """
    return get_completion(message, plot)

def get_bgnumber(para):
    message = f"""
    You will be given a part of a story you have to read that part and have to infere the probable and most suitable location/surrounding in which it's taking place. 
    
    You can give the output out of only the below options provided. You have to give the corresponding integer value from 1 to 15 depending on the location/surrounding you have infered:
    City_night  ---->  1  
    City_day  ---->    2 
    Forest_night  ----> 3
    Forest_day  ---->  4
    Sea_night  ----> 5
    Sea_day  ----> 6
    Room_night  ----> 7
    Room_day  ---->  8
    Village_night  ----> 9
    Village_day  ----> 10
    Moutain_night  ----> 11
    Moutain_day  ----> 12
    Sky_Space_Universe  ----> 13

    You have to strictly follow the given rules:
    1. Always give the output as an integer between 1 to 13 no extra string or anything just an integer number between 1 to 15.
    2. If you are able to infere the location but not able to infere whether it's day or night by default choose the option with day which is an even number \
    for eg. 2, 4, 6, 8, 10, and 12 all these options have day in it so choose out of these options based on the location you have infered.
    3. If the location/surrounding isn't obvious from the text provided then by default give output as 2.

    Here are some examples on how to generate the required option:
    
    Example story paragraph:
    In the heart of a bustling metropolis, where skyscrapers kissed the sky and streets hummed with the \
    rhythm of life, there existed a city like no other. Its streets were a labyrinth of winding alleys and bustling boulevards, \
    lined with towering buildings that reached for the clouds.
    
    As from the above paragraph we can infer that the surrounding is of city and since the time of the day isn't obvious whether \
    it's day or night so you have to select day by default so the final surrounding infered is "City_day" so expected output generated from you is:
    2

    Example story paragraph:
    As the days turned into weeks, Maya forged friendships with the creatures of the jungle. She shared moments of laughter with mischievous monkeys, \
    and learned the ancient wisdom of wise old elephants. Together, they explored hidden caves and winding rivers, each new discovery fueling Maya's sense of wonder.
    
    As from the above paragraph we can infer that the surrounding is of forest/jungle and since the time of the day isn't obvious whether \
    it's day or night so you have to select day by default so the final surrounding infered is "Forest_day" so expected output generated from you is:
    4

    Example story paragraph:
    Once upon a time, in a not-so-distant future, there lived a man named Alex. Alex was an adventurous soul who dreamed of exploring the great \
    unknown: outer space. From a young age, he would gaze up at the stars with wonder, imagining what it would be like to journey among them.

    As from the above paragraph we can infer that the surrounding is of Space/Sky/Universe so the expected output from you is:
    13

    Example story paragraph:
    Under the shimmering canopy of a star-studded night sky, a young boy named Ethan ventures to the edge of the sea. The moon casts a soft, silvery \
    glow upon the restless waves, inviting him into their mysterious depths. With bare feet sinking into cool, wet sand, Ethan hesitates for a moment \
    before plunging into the salty embrace of the ocean. The water is surprisingly warm, and he feels a surge of exhilaration as he dives beneath the \
    surface. Bioluminescent creatures sparkle like underwater stars, illuminating his path as he swims further out. With each stroke, he feels a \
    sense of freedom and wonder, lost in the magic of the nocturnal sea. Laughter echoes in the darkness as Ethan dances with the waves, \
    creating memories that will linger long after the night fades into dawn.

    As from the above paragraph we can infer that the surrounding is of Sea/ocean and the time is of night so the expected output from you is of Sea_night:
    5

    
    Example story paragraph:
    what's up
    Since the location/surrounding to infere is difficult for the above text, you can assume that a person is saying what's up to another person, for this the expected \
    number generated by you is:
    2

    Now the actual story paragraph for which the number is to be generated is the text delimited by triple backticks
    Text:
    ```{para}```
    """
    return get_completion(message, "10")

In [4]:
class Img2ImgModel:
    """
    This class creates a model for transforming images based on given prompts.
    """

    def __init__(
        self,
        model_id_or_path: str,
        device: str = "xpu",
        torch_dtype: torch.dtype = torch.bfloat16,
        optimize: bool = True,
        warmup: bool = False,
        scheduler: bool = True,
    ) -> None:
        """
        Initialize the model with the specified parameters.

        Args:
            model_id_or_path (str): The ID or path of the pre-trained model.
            device (str, optional): The device to run the model on. Defaults to "xpu".
            torch_dtype (torch.dtype, optional): The data type to use for the model. Defaults to torch.float16.
            optimize (bool, optional): Whether to optimize the model. Defaults to True.
        """
        self.device = device
        self.data_type = torch_dtype
        self.scheduler = scheduler
        self.generator = torch.Generator()  # .manual_seed(99)
        self.pipeline = self._load_pipeline(model_id_or_path, torch_dtype)
        if optimize:
            start_time = time.time()
            #print("Optimizing the model...")
            self.optimize_pipeline()
            #print(
            #    "Optimization completed in {:.2f} seconds.".format(
            #        time.time() - start_time
            #    )
            #)
        if warmup:
            self.warmup_model()

    def _load_pipeline(
        self, model_id_or_path: str, torch_dtype: torch.dtype
    ) -> StableDiffusionImg2ImgPipeline:
        """
        Load the pipeline for the model.

        Args:
            model_id_or_path (str): The ID or path of the pre-trained model.
            torch_dtype (torch.dtype): The data type to use for the model.

        Returns:
            StableDiffusionImg2ImgPipeline: The loaded pipeline.
        """
        print("Loading the model...")
        model_path = Path(f"{model_dir}/{model_id_or_path}")
        
        if model_path.exists():
            #print(f"Loading the model from {model_path}...")
            load_path = model_path
        else:
            print("Using the default path for models...")
            load_path = model_id_or_path
            
        pipeline = StableDiffusionImg2ImgPipeline.from_pretrained(
            load_path,
            torch_dtype = torch_dtype,
            use_safetensors = True,
            variant = "fp16",
        )
        if self.scheduler:
            pipeline.scheduler = DPMSolverMultistepScheduler.from_config(
                pipeline.scheduler.config
            )
        if not model_path.exists():
            try:
                print(f"Attempting to save the model to {model_path}...")
                pipeline.save_pretrained(f"{model_path}")
                print("Model saved.")
            except Exception as e:
                print(f"An error occurred while saving the model: {e}. Proceeding without saving.")
        pipeline = pipeline.to(self.device)
        #print("Model loaded.")
        return pipeline

    
    def _optimize_pipeline(self, pipeline: StableDiffusionImg2ImgPipeline) -> StableDiffusionImg2ImgPipeline:
        """
        Optimize the pipeline of the model.

        Args:
            pipeline (StableDiffusionImg2ImgPipeline): The pipeline to optimize.

        Returns:
            StableDiffusionImg2ImgPipeline: The optimized pipeline.
        """

        for attr in dir(pipeline):
            try:
                if isinstance(getattr(pipeline, attr), nn.Module):
                    setattr(
                        pipeline,
                        attr,
                        ipex.optimize(
                            getattr(pipeline, attr).eval(),
                            dtype=pipeline.text_encoder.dtype,
                            inplace=True,
                        ),
                    )
            except AttributeError:
                pass
        return pipeline

    def optimize_pipeline(self) -> None:
        """
        Optimize the pipeline of the model.
        """
        self.pipeline = self._optimize_pipeline(self.pipeline)

    def get_image(self, prompt) -> Image.Image:
        image_number = 10
        try:
            image_number = int(get_bgnumber(prompt))
        except:
            image_number = 10
        if image_number < 1 or image_number > 13:
            image_number = 10
        img = Image.open(f'./base/{image_number}'+'.jpg')
        img = img.resize((512, 300))
        return img

    def generate_images(
        self,
        prompt: str,
        num_inference_steps: int = 200,
        strength: float = 0.75,
        guidance_scale: float = 7.5,
        batch_size: int = 1,
    ):
        time.sleep(22)
        init_image = self.get_image(prompt)
        time.sleep(22)
        try:
            prompt = get_prompt(prompt)
        except:
            prompt = prompt
        image = self.pipeline(
            prompt = prompt,
            image = init_image,
            strength = strength,
            guidance_scale = guidance_scale,
            num_inference_steps=num_inference_steps,
        ).images

        return image[0]


In [5]:
def display_plot(text, image):
    print("okay1")
    plt.figure(figsize=(5, 5))
    Image.imshow(image)
    plt.axis('off')
    plt.show()
    print("okay2")
    print(text)
    print("okay3")

In [6]:
import re

def split_paragraphs(story_text, max_words_per_paragraph = 150):
    if story_text is None:
        story_text = "a man"
        
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', story_text.strip())

    current_paragraph = ''
    paragraphs = []

    for sentence in sentences:
        current_paragraph += sentence.strip() + ' '

        if len(current_paragraph.split()) > max_words_per_paragraph:
            sentences_in_paragraph = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', current_paragraph.strip())

            current_paragraph = ' '.join(sentences_in_paragraph[:-1]).strip()

            paragraphs.append(current_paragraph)

            current_paragraph = sentence.strip() + ' '

    if current_paragraph:
        paragraphs.append(current_paragraph)

    return paragraphs

In [7]:
model_cache = {}

def generate_story():
    out = widgets.Output()
    model_ids = [
        "runwayml/stable-diffusion-v1-5",
        "stabilityai/stable-diffusion-2-1",
    ]
    model_dropdown = widgets.Dropdown(options = model_ids, value = model_ids[0], description = "Select Model:",) 
    prompt_text = widgets.Text(value="", placeholder = "Describe your story in short", description = "Enter Story:", layout = widgets.Layout(width = "600px"))
    
    layout = widgets.Layout(margin = "10px")
    button1 = widgets.Button(description = "Generate Story", button_style = "primary")
    button2 = widgets.Button(description = "Clear Story", button_style = "primary")
    model_dropdown.layout.width = "50%"
    prompt_text.layout.width = "600px"
    button1.layout.margin = "0 0 0 100px"
    button1.layout.width = "150px"
    button2.layout.margin = "0 0 0 300px"
    button2.layout.width = "120px"
    top_row = widgets.HBox([model_dropdown])
    bottom_row = widgets.HBox([prompt_text])
    top_box = widgets.VBox([top_row, bottom_row])
    user_input_widgets = widgets.HBox([top_box], layout = layout)
    bottom_box = widgets.HBox([button1, button2], layout = layout)
    display(user_input_widgets)
    display(bottom_box)
    display(out)
    
   
    
    def generate_image(button):
        clear_output(wait = True)
        print("Creating a new story...")
        story = get_story_from_plot(prompt_text.value)
        partial_stories = split_paragraphs(story)
        for i, parts in enumerate(partial_stories):
            with out:
                button.button_style = "warning"
                selected_model_index = model_ids.index(model_dropdown.value)
                model_id = model_ids[selected_model_index]
                model_key = (model_id, "xpu")
                if model_key not in model_cache:
                    model_cache[model_key] = Img2ImgModel(model_id, device = "xpu")
                prompt = parts
                model = model_cache[model_key]
                if not prompt:
                    prompt = "a man"  
                try:
                    start_time = time.time()
                    image = model.generate_images(prompt = prompt,)
                    image.show()
                    print(prompt)
                except KeyboardInterrupt:
                    print("\nUser interrupted image generation...")
                except Exception as e:
                    print(f"An error occurred: {e}")
                finally:
                    button.button_style = "primary"

    def end_story(button):
        with out:
            clear_output(wait = True)
            print("Creating a new story....")
            
    button1.on_click(generate_image)
    button2.on_click(end_story)

In [8]:
generate_story()

Output()